<a href="https://colab.research.google.com/github/ZHOU-py/NLP/blob/master/%E8%81%8A%E5%A4%A9%E6%9C%BA%E5%99%A8%E4%BA%BA/Cornell_Movies_Dialogs_Corpus_S2S_%E8%81%8A%E5%A4%A9%E6%9C%BA%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 下载数据文件

In [46]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [47]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

#### 加载和预处理数据
- 220,579 conversational exchanges between 10,292 pairs of movie characters
- involves 9,035 characters from 617 movies
- in total 304,713 utterances

In [48]:
corpus_name = "cornell movie-dialogs corpus" 
corpus = os.path.join("data", corpus_name)
def printLines(file, n=10):
  with open(file, 'rb') as datafile:
    lines = datafile.readlines() 
  for line in lines[:n]:
    print(line)
printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


##### 创建格式化数据
解析原始数据文件 movie_lines.txt
* `loadLines`：将文件的每一行拆分为字段（lineID, characterID, movieID, character, text)组合的字典
* `loadConversations`: 根据`movie_conversations.txt`将`loadLines`中的每一行数据进行归类
* `extractSentencePairs`: 从对话中提取句子对


In [49]:
# 将文件的每一行拆分为字段字典
def loadLines(fileName, fields):
  lines = {}
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split(" +++$+++ ")
      lineObj = {}
      for i, field in enumerate(fields):
        lineObj[field] = values[i]
      lines[lineObj['lineID']] = lineObj
  return lines

In [55]:
#  将 'loadLines'中的行字段分组为基于 *movie_conversations.txt* 的对话
def loadConversations(fileName, lines, fields):
  conversations = []
  with open(fileName, 'r', encoding='iso-8859-1') as f:
    for line in f:
      values = line.split(" +++$+++ ")
      # Extract fields
      convObj = {}
      for i, field in enumerate(fields):
        convObj[field] = values[i]
      # Convert string to list (convObj["utteranceIDs"] == "['L598485','L598486', ...]")
      lineIds = eval(convObj["utteranceIDs"])
      # Reassemble lines
      convObj["lines"] = []
#      print(lineIds)
      for lineId in lineIds:
        convObj["lines"].append(lines[lineId])
      conversations.append(convObj)

  return conversations

In [56]:
# 从对话中提取一对句子
def extractSentencePairs(conversations):
  qa_pairs = []
  for conversation in conversations:
    for i in range(len(conversation["lines"])-1):
      inputLine = conversation["lines"][i]["text"].strip()
      targetLine = conversation["lines"][i+1]["text"].strip()
      if inputLine and targetLine:
        qa_pairs.append([inputLine,targetLine])
  return qa_pairs

In [60]:
# 定义新文件的路径
datafile = os.path.join(corpus, "formatted_movie_lines.txt") 
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
# 初始化行dict，对话列表和字段ID
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"] 
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# 加载行和进程对话
print("\n Processing corpus...")
lines = loadLines(os.path.join(corpus,"movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\n Loading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),\
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# 写入新的csv文件
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
  writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n') 
  for pair in extractSentencePairs(conversations):
    writer.writerow(pair)

# 打印一个样本的行
print("\nSample lines from file:") 
printLines(datafile)


 Processing corpus...

 Loading conversations...


KeyError: ignored